In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
import recmetrics

## Load Data

In [2]:
import yaml
from lightgcn import LightGCN
from sklearn.model_selection import train_test_split
from utils import *
pd.set_option('display.max_colwidth', None)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device # = "cpu"

device(type='cpu')

In [4]:
# Load Config
%cd ..
with open("params.yaml") as config_file:
    config = yaml.safe_load(config_file)
config

/Users/yingkang/4thBrain/GNN-eCommerce


{'base': {'random_seed': 42},
 'data': {'cosmetic_shop': 'data/raw/cosmetic-shop-ecommerce-events/',
  'preprocessed': 'data/preprocessed/'},
 'training': {'event_type_weights_v1': {'view': 0.01,
   'cart': 0.1,
   'remove_from_cart': -0.09,
   'purchase': 1.0},
  'event_type_weights_v2': {'view': 0.15,
   'cart': 0.35,
   'remove_from_cart': -0.2,
   'purchase': 1.0}},
 'reports': None}

In [5]:
interaction_matrix = pd.read_csv(config['data']['preprocessed'] + "u_i_weight_0.01_0.1_-0.09.csv")
interaction_matrix

,Unnamed: 0,user_id,item_id,weight
0,0,465496,5769989,0.01
1,1,465496,5865524,0.01
2,2,465496,5865526,0.01
3,3,1120748,5240,0.01
4,4,1180452,5881337,0.01
...,...,...,...,...
10157403,10157403,622090043,5850628,0.01
10157404,10157404,622090052,5688691,0.01
10157405,10157405,622090052,5931986,0.01
10157406,10157406,622090098,5650609,0.01


## Maybe not -- Long Tail Plot for item popularity and active user

In [ ]:
fig = plt.figure(figsize=(15, 7))
recmetrics.long_tail_plot(df=interaction_matrix, 
             item_id_column="item_id", 
             interaction_type="received interests", 
             percentage=0.5,
             x_labels=False)

In [10]:
interaction_matrix['item_id'].value_counts()

5809910    62461
5809912    23741
5751422    21185
5751383    21010
5849033    17383
           ...  
5870513        1
5804500        1
5875090        1
5626579        1
5652262        1
Name: item_id, Length: 54571, dtype: int64

Some item is extremly popular(massive user shows interest to it)

## MAR@K, MAP@K for Baseline model(SVD)

In [ ]:
test = svd_model.test(testSet)
test = pd.DataFrame(test)
test.drop("details", inplace=True, axis=1)
test.columns = ['userId', 'itemId', 'actual', 'SVD_predict']

In [ ]:
actual = test.copy().groupby('userId')['itemId'].agg({
    'actual': (lambda x: list(set(x)))}).reset_index()

In [ ]:
actual = actual.set_index("userId")

In [ ]:
predict = test.pivot_table(index='userId', columns='itemId', values='SVD_predict').fillna(0)

In [ ]:
def get_users_predictions(user_id, K, model):
    recommended_items = pd.DataFrame(model.loc[user_id])
    recommended_items.columns = ["predicted_rating"]
    recommended_items = recommended_items.sort_values('predicted_rating', ascending=False)    
    recommended_items = recommended_items.head(K)
    return recommended_items.index.tolist()

In [ ]:
cf_recs = [] = []
for user in actual.index:
    cf_predictions = get_users_predictions(user, 10, predict)
    cf_recs.append(cf_predictions)

In [ ]:
actual = test.actual.values.tolist()
cf_predictions = test.SVD_predict.values.tolist()

In [ ]:
cf_mark = []
for K in np.arange(1, 11):
    cf_mark.extend([recmetrics.mark(actual, cf_predictions, k=K)])
cf_mark

### MAR@K Plot

In [ ]:
mark_scores = [random_mark, pop_mark, cf_mark]
index = range(1,10+1)
names = ['Random Recommender', 'Popularity Recommender', 'Collaborative Filter']

fig = plt.figure(figsize=(15, 7))
recmetrics.mark_plot(mark_scores, model_names=names, k_range=index)

## Hit Rate

## ROC Plot + Precision Recall Plot

## Average Precision

## Prediction Coverage + Coverage Plot